<a href="https://colab.research.google.com/github/WxAnalyst/Colab-Projects/blob/main/Temperature_Streaks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACIS Cold and Warm 'Streaks'
This notebook is designed to download data from xmacis and output the length of cold and warm streaks (where the average daily temperature is below or above climatological normals). The output will also calculate the number of colder and warmer than normal days in the specified period.

In [ ]:
!pip install siphon

In [2]:
from siphon.simplewebservice.acis import acis_request
#import json
import pandas as pd
import numpy as np

In [68]:
cwa = "PIH" #@param {type:"string"}
start_date = '2023-01-01' #@param {type:"date"}
end_date = '2023-03-31'   #@param {type:"date"}
params = {"cwa": cwa, #"PIH",
          "elems":
            [{"name": "avgt",
              "normal": "departure"}],
          "sdate": start_date, #"2023-01-01"},
          "edate": end_date,   #"2023-03-31",
          "meta": "name,elems"}

ds = acis_request("MultiStnData", params)

In [70]:
# Create and cleanup the dataframe
dj = pd.DataFrame(ds['data'])
dj['meta'] = dj['meta'].apply(lambda x: x['name'])
for i in range(0, len(dj.data)):
  dj.data[i] = [item for sublist in dj.data[i] for item in sublist]

# Gather the longest streaks of warmer than and colder than normal temperatures
cold_streak = []
warm_streak = []
countCold = []
countWarm = []
# Iterate through all stations
for ser in range(0, len(dj['data'])):
  cold = 0
  warm = 0
  count_cold = 0
  count_warm = 0
  max_cold = 0
  max_warm = 0
  # Iterate through values
  for i in range(0, len(dj['data'][ser])):
    if dj.data[ser][i] == 'M':
      continue
    if float(dj.data[ser][i]) < 0:
      cold += 1
      if warm > max_warm:
        max_warm = warm
      warm = 0
      count_cold += 1
    if float(dj.data[ser][i]) > 0:
      warm += 1
      if cold > max_cold:
        max_cold = cold
      cold = 0
      count_warm += 1
    if float(dj.data[ser][i]) == 0:
      if warm > max_warm:
        max_warm = warm
      warm = 0
      if cold > max_cold:
        max_cold = cold
      cold = 0
  cold_streak.append(max_cold)
  warm_streak.append(max_warm)
  countCold.append(count_cold)
  countWarm.append(count_warm)

# Add the streaks and the counts to the dataframe
dj['cold_streak'] = cold_streak
dj['warm_streak'] = warm_streak
dj['count_cold'] = countCold
dj['count_warm'] = countWarm

In [72]:
table = dj[['meta', 'cold_streak','warm_streak','count_cold','count_warm']].copy()

In [73]:
table

,meta,cold_streak,warm_streak,count_cold,count_warm
0,GAS CAVES IDAHO,0,0,0,0
1,MOODY IDAHO,0,0,0,0
2,ISLAND PARK,14,12,64,25
3,ISLAND PARK IDAHO,0,0,0,0
4,ISLAND PARK,0,0,0,0
...,...,...,...,...,...
111,PAUL 1ENE,0,0,0,0
112,MINIDOKA DAM,17,13,65,25
113,RAFT RIVER IDAHO,0,0,0,0
114,HOWELL CANYON,0,0,0,0
